In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.regularizers import L2
from tensorflow.keras.layers import Dense, Dropout, InputLayer


import data_processing as dp

2024-11-26 22:09:12.138094: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-26 22:09:12.139350: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-26 22:09:12.146739: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-26 22:09:12.158893: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1732621152.175845  724372 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1732621152.18

In [2]:
# Load dataset
dataset_name = "output_dataset.csv"
df = pd.read_csv(dataset_name)

In [3]:
# Process dataset
df = dp.add_headers(df)
df = dp.trim_id(df)
df = dp.mask_data(df)
df = dp.impute_nans(df)
df

/home/tom/personal-projects/Yahtzee-Tools/server/data_processing.py:54: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: np.random.rand() if pd.isna(x) else x)


,Ones,Twos,Threes,Fours,Fives,Sixes,Total,Bonus,3 of a Kind,4 of a Kind,...,Total Mask,Bonus Mask,3 of a Kind Mask,4 of a Kind Mask,Full House Mask,Small Straight Mask,Large Straight Mask,Chance Mask,Yahtzee Mask,Score Mask
0,0.764849,0.098236,0.182197,0.729897,0.861138,0.702684,0.267828,0.462392,0.504530,0.625953,...,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0
1,0.335765,0.204440,0.154620,0.237302,0.884010,18.000000,0.852621,0.243772,0.115154,0.085343,...,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0
2,0.121841,0.479616,0.778028,0.200332,0.551002,18.000000,0.296121,0.923280,25.000000,0.912261,...,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0
3,0.327652,0.357228,0.667100,12.000000,0.186080,18.000000,0.277276,0.266191,25.000000,0.752854,...,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0
4,4.000000,0.485123,0.933361,12.000000,0.206290,18.000000,0.569239,0.716068,25.000000,0.115904,...,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1524,2.000000,4.000000,0.619435,12.000000,20.000000,24.000000,0.489671,0.933913,0.541004,0.323255,...,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
1525,2.000000,4.000000,0.040223,12.000000,20.000000,24.000000,0.967810,0.537200,24.000000,0.489966,...,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
1526,2.000000,4.000000,0.204752,12.000000,20.000000,24.000000,0.167464,0.194301,24.000000,0.903931,...,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1527,2.000000,4.000000,0.493386,12.000000,20.000000,24.000000,0.472183,0.372947,24.000000,16.000000,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
# Split dataset
X, y = dp.split_xy(df)
X_train, X_test, y_train, y_test = dp.split_train_test(X, y, 0.2)

# Normalize data
X_train = dp.scale_data(X_train, True)
X_test = dp.scale_data(X_test, False)

y

0       271
1       271
2       271
3       271
4       271
       ... 
1524    312
1525    312
1526    312
1527    312
1528    312
Name: Score, Length: 1529, dtype: int64

In [5]:
l2_reg = L2(0.01)  # Regularization factor (lambda)

# Deep Neural Network
model = Sequential([
    InputLayer(shape=(31,)),
    Dense(64, activation='relu', kernel_regularizer=l2_reg),
    Dense(32, activation='relu', kernel_regularizer=l2_reg),
    Dense(16, activation='relu', kernel_regularizer=l2_reg),
    Dense(1)
])

2024-11-26 22:09:14.563244: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [6]:
# Compile the model
model.compile(optimizer='adam', loss='mean_absolute_error')

In [7]:
# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=16, validation_split=0.1)

Epoch 1/50
69/69 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 249.3739 - val_loss: 211.0728
Epoch 2/50
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 194.7065 - val_loss: 79.2562
Epoch 3/50
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 72.8443 - val_loss: 55.2917
Epoch 4/50
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 45.9030 - val_loss: 46.3960
Epoch 5/50
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 38.4179 - val_loss: 40.6047
Epoch 6/50
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 35.1017 - val_loss: 39.2186
Epoch 7/50
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 32.4499 - val_loss: 38.9822
Epoch 8/50
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 31.9962 - val_loss: 37.5071
Epoch 9/50
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 31.5650 - val_loss: 37.4580
Epoch 10/50
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 29.9531 - val_loss: 37.6822
Epoch 11/50
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 31.5250 - val_loss: 39.9974
Epoch 12/50
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/ste

In [11]:
# Test loss
test_loss = model.evaluate(X_test, y_test)
print(f"Test Loss: {test_loss}")

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 27.9660
Test Loss: 27.425853729248047


In [19]:
import random

# Make predictions on the test data, print random
random_index = random.randint(0, len(X_test) - 1)
predictions = model.predict(X_test)
print("Data: ", list(X_test[random_index]))
print("Prediction: ", predictions[random_index][0]) 
print("Actual: ", y_test.iloc[random_index])

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
Data:  [-0.3977190919278672, 0.8943287619643965, 1.5222345140446012, 0.8625911897103202, 0.8088059833494358, 0.7097742696907076, -0.42430754936254794, -0.3510774140818635, -0.9030054665623026, -0.6338940570193567, 0.753533830617963, 0.7310465055323538, 1.2209055739532544, 1.0395547845643076, 0.9002912738179939, 1.3953451013305738, -0.9025187307458946, -1.0581039908531071, -1.0409011951794773, -1.007386230857813, -0.962280755272859, 0.4487485326767854, 0.4487485326767854, 0.9466382788624977, -1.2106716344457786, -0.7219032335479355, -0.7284664228098997, -1.115363930404649, -0.9205607118322965, -1.1725004398947565, 0.0]
Prediction:  279.35977
Actual:  292


In [ ]:
from math import nan

# Custom tests
x1 = nan;   x2 = nan;   x3 = nan
x4 = nan;   x5 = nan;   x6 = nan
tt = nan;   bn = nan;   k3 = nan
k4 = nan;   fh = nan;   ss = nan
ls = nan;   ch = nan;   yz = nan

sample = ["Sample",x1,x2,x3,x4,x5,x6,tt,bn,k3,k4,fh,ss,ls,ch,yz,0]
sample_df = pd.DataFrame([sample])
sample_df = dp.add_headers(sample_df)
sample_df = dp.trim_id(sample_df)
sample_df = dp.mask_data(sample_df)
sample_df = dp.impute_nans(sample_df)
sample_df, _ = dp.split_xy(sample_df)
sample_df = dp.scale_data(sample_df, False)

model.predict(sample_df)[0][0]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step


/home/tom/personal-projects/Yahtzee-Tools/server/data_processing.py:54: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: np.random.rand() if pd.isna(x) else x)


266.0704

In [ ]:
h5_filename = "output_model.keras"
model.save(h5_filename)